# TensorBoard Embedding

https://www.tensorflow.org/get_started/embedding_viz

    tensorboard --logdir=/data/podondra/embedding

In [1]:
import h5py
import numpy as np
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

In [2]:
# open HDF5 file with all data
f = h5py.File('data.hdf5')
X = np.array(f['X_train'], dtype=np.float32)
y = f['y_train'][...]

In [3]:
X.shape, y.shape

((1547, 4000), (1547,))

In [4]:
LOG_DIR = '/data/podondra/embedding/'

In [5]:
sess = tf.InteractiveSession()

In [6]:
if tf.gfile.Exists(LOG_DIR):
    tf.gfile.DeleteRecursively(LOG_DIR)
tf.gfile.MakeDirs(LOG_DIR)
summary_writer = tf.summary.FileWriter(LOG_DIR, sess.graph)

In [7]:
# save metadata
np.savetxt(LOG_DIR + 'metadata.tsv', y.reshape(-1, 1), fmt='%d')

In [8]:
config = projector.ProjectorConfig()

In [9]:
def add_embedding(X, name):
    embedding_var = tf.Variable(tf.constant(X), name=name)
    embedding = config.embeddings.add()
    embedding.tensor_name = embedding_var.name
    # link this tensor to its metadata file
    embedding.metadata_path = LOG_DIR + 'metadata.tsv'
    
add_embedding(X, 'normalized')
add_embedding((X - 1) / np.max(np.abs(X - 1)), 'scaled')

In [10]:
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()
saver.save(sess, LOG_DIR + 'model.ckpt', 0)

'/data/podondra/embedding/model.ckpt-0'

In [11]:
projector.visualize_embeddings(summary_writer, config)